In [14]:
install.packages("dplyr", repos="http://cran.rstudio.com/") 
install.packages("RSQLite", repos="http://cran.rstudio.com/") 
install.packages("DBI", repos="http://cran.rstudio.com/") 
install.packages("sqldf", repos="http://cran.rstudio.com/") 


Installing package into 'C:/Users/tsankhe/Documents/R/win-library/3.3'
(as 'lib' is unspecified)
Warning message:
"package 'dplyr' is in use and will not be installed"Installing package into 'C:/Users/tsankhe/Documents/R/win-library/3.3'
(as 'lib' is unspecified)
Warning message:
"package 'RSQLite' is in use and will not be installed"Installing package into 'C:/Users/tsankhe/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'DBI' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\tsankhe\AppData\Local\Temp\Rtmp4YxTQk\downloaded_packages


In [4]:
library(dplyr)
library(DBI)
library(RSQLite)
library(sqldf)

Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'DBI' was built under R version 3.3.3"

In [5]:
dbfile<-"database.sqlite"

In [6]:
dbcon <- dbConnect(dbDriver("SQLite"),dbname = dbfile)

In [22]:
tablesAll = dbListTables(dbcon)
print(tablesAll)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"  


In [28]:
player.df = dbGetQuery(dbcon,'select * from Player' )
team.df = dbGetQuery(dbcon,'select * from Team' )
playerAttri.df = dbGetQuery(dbcon,'select * from Player_Attributes' )

In [29]:
head(player.df)
head(playerAttri.df)


id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161


id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,...,54,48,65,66,69,6,11,10,8,8
4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
5,218353,505942,2007-02-22 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
6,189615,155782,2016-04-21 00:00:00,74,76,left,high,medium,80,...,66,59,76,75,78,14,7,9,9,12


### data munging to compute overall statistics using sql addition of columns and aliasing

In [30]:
playerStats<-dbGetQuery(dbcon, "SELECT player_api_id, player_name, date, overall_rating,(gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes) AS gk_overall FROM Player_Attributes INNER JOIN Player USING(player_api_id)")

In [31]:
nrow(playerStats)

[1] 183978

In [32]:
teamStats<-dbGetQuery(dbcon, "SELECT team_api_id, team_long_name, date,  (buildUpPlaySpeed+ buildUpPlayDribbling+ buildUpPlayPassing) AS build_up_play, (chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting) AS chance_creation, ( defencePressure+defenceAggression+defenceTeamWidth ) AS defence FROM Team_Attributes INNER JOIN Team USING(team_api_id)")

In [33]:
head(teamStats)

team_api_id,team_long_name,date,build_up_play,chance_creation,defence
9930,FC Aarau,2010-02-22 00:00:00,NA,180,150
9930,FC Aarau,2014-09-19 00:00:00,156,181,145
9930,FC Aarau,2015-09-10 00:00:00,142,181,145
8485,Aberdeen,2010-02-22 00:00:00,NA,210,200
8485,Aberdeen,2011-02-22 00:00:00,NA,153,146
8485,Aberdeen,2012-02-22 00:00:00,NA,170,140


In [34]:
teamStats[is.na(teamStats)]<-0

In [35]:
head(teamStats)

team_api_id,team_long_name,date,build_up_play,chance_creation,defence
9930,FC Aarau,2010-02-22 00:00:00,0,180,150
9930,FC Aarau,2014-09-19 00:00:00,156,181,145
9930,FC Aarau,2015-09-10 00:00:00,142,181,145
8485,Aberdeen,2010-02-22 00:00:00,0,210,200
8485,Aberdeen,2011-02-22 00:00:00,0,153,146
8485,Aberdeen,2012-02-22 00:00:00,0,170,140


In [36]:
match.df<-dbGetQuery(dbcon, "SELECT * FROM Match")

### obtain relevant rows from Match table, remove 'NA' values. 
### Compute count for each player in matches he has played using unlist and grep, counting across all colums and rows for the number of times a player appears.

In [37]:
match.df<-match.df[, 56:77]
head(match.df)

home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [38]:
match.df<-na.omit(match.df)

In [39]:
head(match.df)

,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
146,38327,67950,67958,67959,37112,36393,148286,67898,164352,38801,...,38293,148313,104411,148314,37202,43158,9307,42153,32690,38782
154,36835,37047,37021,37051,104386,32863,37957,37909,38357,37065,...,21812,11736,37858,38366,37983,39578,38336,52280,27423,38440
156,34480,38388,26458,13423,38389,30949,38393,38253,38383,38778,...,37886,37903,37889,94030,37893,37981,131531,130027,38231,131530
163,38327,67950,67958,38801,67898,37112,67959,148286,164352,33657,...,38388,38389,31316,164694,30949,38378,38383,38393,38253,37069
169,37900,37886,37100,37903,37889,37893,37981,131531,131530,38231,...,38247,16387,94288,94284,45832,26669,33671,163670,37945,33622
174,104378,38248,36841,38255,104382,33662,37044,32760,38229,39875,...,37887,37886,37903,94030,37981,131531,131530,130027,38231,75500


In [40]:
matchStats<-as.data.frame(table(unlist(match.df[grep("",names(match.df))])))
nrow(matchStats)
head(matchStats)

[1] 10787

Var1,Freq
2625,4
2752,79
2768,69
2770,14
2790,40
2796,1


### merge computed appearances with player table.

In [41]:
playerStats<-merge(x=playerStats, y=matchStats, by.x="player_api_id", by.y="Var1")

In [42]:
head(playerStats)

player_api_id,player_name,date,overall_rating,gk_overall,Freq
2625,"Patryk Rachwal,18",2012-02-22 00:00:00,58,45,4
2625,"Patryk Rachwal,18",2013-03-08 00:00:00,61,45,4
2625,"Patryk Rachwal,18",2011-02-22 00:00:00,59,45,4
2625,"Patryk Rachwal,18",2014-09-18 00:00:00,61,45,4
2625,"Patryk Rachwal,18",2012-08-31 00:00:00,58,45,4
2625,"Patryk Rachwal,18",2014-11-07 00:00:00,61,45,4


In [43]:
names(playerStats)[names(playerStats) == 'Freq'] <- 'number_of_appearances'
head(playerStats)

player_api_id,player_name,date,overall_rating,gk_overall,number_of_appearances
2625,"Patryk Rachwal,18",2012-02-22 00:00:00,58,45,4
2625,"Patryk Rachwal,18",2013-03-08 00:00:00,61,45,4
2625,"Patryk Rachwal,18",2011-02-22 00:00:00,59,45,4
2625,"Patryk Rachwal,18",2014-09-18 00:00:00,61,45,4
2625,"Patryk Rachwal,18",2012-08-31 00:00:00,58,45,4
2625,"Patryk Rachwal,18",2014-11-07 00:00:00,61,45,4


### count number of league's a player has played in, using group by clause of sql 

In [44]:
join<- dbGetQuery(dbcon, "SELECT COUNT(DISTINCT(league_id)) AS number_of_leagues, player_api_id FROM Player INNER JOIN Match WHERE (player_api_id=home_player_1 OR player_api_id=home_player_2 OR player_api_id=home_player_3 OR player_api_id=home_player_4 OR player_api_id= home_player_5 OR player_api_id= home_player_6 OR player_api_id= home_player_7 OR player_api_id=home_player_8 OR player_api_id=home_player_9 OR player_api_id=home_player_10 OR player_api_id=home_player_11 OR player_api_id=away_player_1 OR player_api_id=away_player_2 OR player_api_id=away_player_3 OR player_api_id=away_player_4 OR player_api_id=away_player_5 OR player_api_id=away_player_6 OR player_api_id=away_player_7 OR player_api_id=away_player_8 OR player_api_id=away_player_9 OR player_api_id=away_player_10 OR player_api_id=away_player_11)  GROUP BY player_api_id")

In [45]:
head(join)

number_of_leagues,player_api_id
1,2625
1,2752
1,2768
2,2770
1,2790
1,2796


In [46]:
playerStats<-merge(x=playerStats, y=join, by.x="player_api_id", by.y="player_api_id")

In [47]:
head(playerStats)

player_api_id,player_name,date,overall_rating,gk_overall,number_of_appearances,number_of_leagues
2625,"Patryk Rachwal,18",2012-02-22 00:00:00,58,45,4,1
2625,"Patryk Rachwal,18",2013-03-08 00:00:00,61,45,4,1
2625,"Patryk Rachwal,18",2011-02-22 00:00:00,59,45,4,1
2625,"Patryk Rachwal,18",2014-09-18 00:00:00,61,45,4,1
2625,"Patryk Rachwal,18",2012-08-31 00:00:00,58,45,4,1
2625,"Patryk Rachwal,18",2014-11-07 00:00:00,61,45,4,1


### For number of goals map home goals and aways goals to respective teams seperately and then merge the two views using team_api_id value later.

In [48]:
goals<-dbGetQuery(dbcon, "Select home_team_api_id, away_team_api_id,home_team_goal, away_team_goal from Match")
head(goals)

home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
9987,9993,1,1
10000,9994,0,0
9984,8635,0,3
9991,9998,5,0
7947,9985,1,3
8203,8342,1,1


In [49]:
homeGoals<-sqldf("Select sum(home_team_goal) as home, home_team_api_id from goals group by home_team_api_id") 
awayGoals<-sqldf("Select sum(away_team_goal) as away, away_team_api_id from goals group by away_team_api_id")
head(homeGoals)
head(awayGoals)

home,home_team_api_id
148,1601
74,1773
176,1957
70,2033
215,2182
96,2183


away,away_team_api_id
132,1601
52,1773
111,1957
68,2033
157,2182
88,2183


In [50]:
totalGoals<-sqldf("select home_team_api_id as team_id, (home+away) as total_goals from homeGoals inner join awayGoals where home_team_api_id=away_team_api_id")

In [51]:
head(totalGoals)

team_id,total_goals
1601,280
1773,126
1957,287
2033,138
2182,372
2183,184


In [52]:
teamStats<-merge(x=teamStats, y=totalGoals, by.x="team_api_id", by.y="team_id")

In [54]:
head(teamStats)
head(playerStats)

team_api_id,team_long_name,date,build_up_play,chance_creation,defence,total_goals
1601,Ruch Chorzów,2010-02-22 00:00:00,0,155,175,280
1601,Ruch Chorzów,2011-02-22 00:00:00,0,186,144,280
1601,Ruch Chorzów,2012-02-22 00:00:00,0,159,136,280
1601,Ruch Chorzów,2013-09-20 00:00:00,0,159,136,280
1601,Ruch Chorzów,2014-09-19 00:00:00,139,181,136,280
1601,Ruch Chorzów,2015-09-10 00:00:00,133,176,136,280


player_api_id,player_name,date,overall_rating,gk_overall,number_of_appearances,number_of_leagues
2625,"Patryk Rachwal,18",2012-02-22 00:00:00,58,45,4,1
2625,"Patryk Rachwal,18",2013-03-08 00:00:00,61,45,4,1
2625,"Patryk Rachwal,18",2011-02-22 00:00:00,59,45,4,1
2625,"Patryk Rachwal,18",2014-09-18 00:00:00,61,45,4,1
2625,"Patryk Rachwal,18",2012-08-31 00:00:00,58,45,4,1
2625,"Patryk Rachwal,18",2014-11-07 00:00:00,61,45,4,1


### write the dataFrames to csv files.

In [56]:
write.csv(file="Players.csv", playerStats)
write.csv(file="Teams.csv", teamStats)

### References

#### https://www.w3schools.com/sql/
#### https://cran.r-project.org/web/packages/RSQLite/RSQLite.pdf
#### https://stat.ethz.ch/R-manual/R-devel/library/base/html/merge.html
#### https://stat.ethz.ch/R-manual/R-devel/library/base/html/grep.html
#### https://stat.ethz.ch/R-manual/R-devel/library/base/html/unlist.html
#### https://www.tutorialspoint.com/sqlite/
